<a href="https://colab.research.google.com/github/HaroldSthid/geospatialanalysis/blob/main/GeoSpatialAnalysis_AluraAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proceso de Geocodificacion

Release v1.38.1. (Installation) Simple and consistent geocoding library written in Python.

In [ ]:
!pip install geocoder tqdm

#Probando otras liberias de Geocodificacion

##Geopy
Geopy es una biblioteca de Python que facilita la localización de las coordenadas de direcciones, ciudades y países a través de geocodificadores y otros servicios de datos geoespaciales. Permite a los usuarios obtener la ubicación geográfica de una dirección mediante la geocodificación y también ofrece la funcionalidad de geocodificación inversa.

In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode("175 5th Avenue NYC")
print(location.latitude, location.longitude)

##ArcGIS
ArcGIS ofrece una API de geocodificación que se puede utilizar a través de la biblioteca de Python.

In [ ]:
!pip install arcgis

In [ ]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS

###Step 1: Instalar la biblioteca arcgis y montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Step 2: Leer el archivo xlsx en un DataFrame de pandas

In [ ]:
import pandas as pd

# Asegúrate de tener la ruta correcta a tu archivo xlsx aquí
file_path = '/content/drive/My Drive/Geocoding/tbl_pagespdf_unificada.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

###Step 3: Geocodificar las direcciones

In [ ]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from tqdm.auto import tqdm

In [ ]:
# Iniciar sesión en ArcGIS si es necesario
# gis = GIS("https://www.arcgis.com", "tu_usuario", "tu_contraseña")

# Usaremos la API de ArcGIS sin autenticación para este ejemplo
gis = GIS()

# Función para geocodificar una dirección
def geocodificar_direccion(direccion):
    try:
        return geocode(address=direccion)[0]['location']
    except Exception as e:
        print(f"Error al geocodificar {direccion}: {e}")
        return None

# Agregar columnas para las coordenadas
df['Latitud'] = None
df['Longitud'] = None

# Geocodificar las direcciones y actualizar el DataFrame
for index in tqdm(df.index, desc='Geocodificando'):
    direccion = df.loc[index, 'Direccion'] + ', ' + df.loc[index, 'Ciudad'] + ', ' + df.loc[index, 'Departamento']
    resultado_geocodificacion = geocodificar_direccion(direccion)
    if resultado_geocodificacion:
        df.at[index, 'Latitud'] = resultado_geocodificacion['y']
        df.at[index, 'Longitud'] = resultado_geocodificacion['x']

###Step 4: Guardar los resultados

In [ ]:
# Ruta para guardar el archivo actualizado en Google Drive
output_file_path = '/content/drive/My DriveGeocoding/output_geocoding_tbl_pagespdf_unificada.xlsx'
df.to_excel(output_file_path, index=False)

# Imprimir la ruta del archivo de salida
print(f"Archivo actualizado guardado en: {output_file_path}")

Visualizar coordenadas en un mapa usando Folium en Google Colab, primero debemos asegurarnos de que Folium esté instalado y luego generar el mapa con las ubicaciones basadas en las coordenadas obtenidas anteriormente. Vamos a proceder paso a paso

###Step 5: Instalar Folium en Colab

In [ ]:
!pip install folium

###Step 6: Preparar los Datos Después de geocodificar

 las direcciones como en los pasos anteriores, asegúrate de que tu DataFrame df tiene las columnas 'Latitud' y 'Longitud' con las coordenadas correctas.

In [ ]:
# Asegúrate de tener la ruta correcta al archivo .xlsx
file_path = '/content/drive/My Drive/Geocoding/output_geocoding_tbl_pagespdf_unificada.xlsx'

# Leer el archivo en un DataFrame de pandas
df = pd.read_excel(file_path, engine='openpyxl')

###Step7: Crear un Mapa con Folium

In [ ]:
# Crear un mapa base usando la media de las coordenadas
mapa = folium.Map(location=[df['Latitud'].mean(), df['Longitud'].mean()], zoom_start=12)

# Añadir marcadores para cada ubicación en el DataFrame
for idx, row in df.iterrows():
    # Asegúrate de que las filas tienen coordenadas válidas
    if not pd.isnull(row['Latitud']) and not pd.isnull(row['Longitud']):
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=f"{row['Nombre Establecimiento']}", # Asegúrate de que esta columna existe en tu DataFrame
        ).add_to(mapa)

# Mostrar el mapa en el notebook de Colab
mapa

###Step 8: Guardar el Mapa en un Archivo HTML (Opcional)
Si deseas guardar el mapa en un archivo HTML para abrirlo con un navegador web o compartirlo con otros, puedes hacer lo siguiente:

In [ ]:
# Guardar el mapa en un archivo HTML
output_map_path = '/content/drive/My Drive/Geocoding/mapa_sucursal_NAC.html'
mapa.save(output_map_path)

# Imprimir la ruta del archivo del mapa
print(f"Mapa guardado en: {output_map_path}")

#Iteracion Clusters con DBSCAN

###Step 9: Instalación de dependencias y montaje de Google Drive

In [ ]:
!pip install folium scikit-learn openpyxl

In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
import folium
from folium.plugins import MarkerCluster

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Step 10: Leer el archivo .xlsx en un DataFrame

In [ ]:
# Asegúrate de reemplazar 'path_to_your_file.xlsx' con la ruta real de tu archivo en Google Drive
file_path = '/content/drive/My Drive/Geocoding/output_geocoding_tbl_pagespdf_unificada.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

###Step 11: Aplicar DBSCAN para clustering basado en densidad

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
# Transformar las coordenadas a radianes para el algoritmo DBSCAN
coords = df[['Latitud', 'Longitud']].to_numpy()
coords_radians = np.radians(coords)

# Aplicar DBSCAN - Ajusta estos parámetros según sea necesario
db = DBSCAN(eps=0.1/6371., min_samples=5, algorithm='ball_tree', metric='haversine')
df['Cluster'] = db.fit_predict(coords_radians)

###Step 12: Crear el mapa con Folium y visualizar clusters y etiquetas

In [ ]:
# Crear un mapa base
mapa = folium.Map(location=[df['Latitud'].mean(), df['Longitud'].mean()], zoom_start=12)

# Crear un MarkerCluster para agrupar los marcadores
marker_cluster = MarkerCluster().add_to(mapa)

# Añadir marcadores con etiquetas que incluyen el nombre y la dirección
for idx, row in df.iterrows():
    if not pd.isnull(row['Latitud']) and not pd.isnull(row['Longitud']):
        etiqueta = f"{row['Nombre Establecimiento']} - {row['Direccion']}"
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=etiqueta,
            icon=folium.Icon(color='blue' if row['Cluster'] != -1 else 'red')
        ).add_to(marker_cluster)

# Trazar polígonos alrededor de los clusters
for cluster_number in np.unique(df['Cluster']):
    cluster_data = df[df['Cluster'] == cluster_number]

    if cluster_number != -1 and len(cluster_data) > 2:
        locations = cluster_data[['Latitud', 'Longitud']].values.tolist()
        folium.vector_layers.Polygon(locations=locations,
                                     color='green',
                                     fill=True,
                                     fill_color='green',
                                     fill_opacity=0.2).add_to(mapa)

# Mostrar el mapa
mapa